## Load Embeddings Model

In [1]:
from sentence_transformers import SentenceTransformer
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

/home/ubuntu/.local/share/virtualenvs/llm-homework-03-TuzjjQaC/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [18]:
q_v = embedding_model.encode(user_question)
print(q_v[0])

0.07822264


## Prepare the Documents

In [12]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
print(documents[0])

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp', 'id': 'c02e79ef'}


In [17]:
# Filter the docs on just the ML course
documents = [doc for doc in documents if doc['course'] == "machine-learning-zoomcamp"]
print(len(documents))

375


## Create the Embeddings

In [19]:
import numpy as np

In [22]:
from tqdm.auto import tqdm

In [24]:
embeddings = []

for doc in tqdm(documents):
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings.append(embedding_model.encode(qa_text))

X = np.array(embeddings)
print(X.shape)

100%|██████████| 375/375 [00:27<00:00, 13.57it/s]

(375, 768)


## Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the question vector and the matrix X.

Since the vectors returned from the embedding model are already normalized, we can use the dot product to compute cosine similarity. 

In [37]:
scores = X.dot(q_v)
scores.sort()
print(scores[-1]) # The top score

0.6506573


## Vector Search

In [ ]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings # matrix, each row is a vector representation of the docs

    # Searches for most similar documents to the v_query
    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query) # Computes similarity score
        idx = np.argsort(-scores)[:num_results] # Sort scores in desc order, get top indices (up to num results)
        return [self.documents[i] for i in idx] # Return top docs

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(q_v, num_results=5)